# Load Libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_auc_score, confusion_matrix, f1_score,\
                            precision_score, recall_score, accuracy_score,\
                            average_precision_score, precision_recall_curve,\
                            matthews_corrcoef
from imblearn.over_sampling import RandomOverSampler
from src.helper_functions import load_data, get_model_perfs, init_model_perfs,\
                                 save_model, save_model_perfs, check_is_best,\
                                 read_model, evaluate_model_predictions,\
                                 update_model_perfs, check_and_save,\
                                 adjusted_classes
import pickle
import os
import csv
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from timeit import default_timer as timer
import keras
from keras.callbacks import Callback
from keras.layers import Dense, Activation, Dropout
keras.__version__

Using TensorFlow backend.


'2.2.4'

## References:
* https://hyperopt.github.io/hyperopt/
* https://towardsdatascience.com/an-introductory-example-of-bayesian-optimization-in-python-with-hyperopt-aae40fff4ff0
* https://towardsdatascience.com/automated-machine-learning-hyperparameter-tuning-in-python-dfda59b72f8a
* https://towardsdatascience.com/a-conceptual-explanation-of-bayesian-model-based-hyperparameter-optimization-for-machine-learning-b8172278050f

### Callback Class
See https://www.kaggle.com/inspector/keras-hyperopt-example-sketch/log
if I need this again.

## Continue Search? (If trials data exists)
Helpful reference: https://github.com/hyperopt/hyperopt/issues/267

In [2]:
Continue_Existing_Trials = True

# Load Data
To obtain source data files:
>cd ./data/raw  
curl http://bioinf.jku.at/research/DeepTox/tox21.zip --output tox21.zip  
unzip tox21.zip -d tox21

In [3]:
x_tr, y_tr, x_te, y_te = load_data()

# Function for Setting up Tracking File
This allows for monitoring progress even while optimization is running.  It is not the same as the trials object which keeps track of optimization progress details.

In [4]:
def set_up_tracking_file(out_file):
    # File to save first results
    of_connection = open(out_file, 'w')
    writer = csv.writer(of_connection)

    # Write the headers to the file
    writer.writerow(['loss', 'params', 'iteration', 'train_time'])
    of_connection.close()

# Define Optimization Space

In [5]:
space = {'layers': hp.choice('layers', range(1,5)),
         'drop_out': hp.uniform('drop_out', 0, 0.5),
         'L2_reg': hp.choice('L2_reg', [0.01, 0.001, 0.0001, 0.0001]),
         'act': hp.choice('act', ['sigmoid', 'tanh']),
         'neurons': hp.choice('neurons', [64, 128, 256, 512, 1024]),
         'decay': hp.choice('decay', [0, 10**-6, 10**-5, 10**-4]),
         'learn_rate': hp.choice('learn_rate', [0.001, 0.01, 0.1, 1, 10 ]),
         'batch_size': hp.choice('batch_size', [16, 32, 64, 128, 256])
        }

# Define Objective Function

In [6]:
def objective(params):
    # Save current state of trials object so we can recover from issues
    pickle.dump(trials, open(os.path.join(target_path,'trials.pkl'), "wb"))
                             
    # Keep track of evals
    global ITERATION
    ITERATION += 1
    
    # Build Dense Neural Network
    DNN = keras.Sequential()
    DNN.add(Dense(params['neurons'], activation=params['act'],input_shape=x_tr.shape[1:],name='h0_'+params['act']+'_activation'))
    DNN.add(Dropout(rate=params['drop_out'],name='Dropout0'))
    for i in range(1,params['layers']):
        DNN.add(Dense(units=params['neurons'], activation=params['act'],\
                      name='h'+str(i)+'_'+params['act']+'_activation',\
                      kernel_regularizer=keras.regularizers.l2(params['L2_reg'])))
        DNN.add(Dropout(rate=params['drop_out'],name='Dropout'+str(i)))
    DNN.add(Dense(units=1, activation='sigmoid'))
    keras.optimizers.Adam(lr=params['learn_rate'], beta_1=0.9,\
                          beta_2=0.999, decay=params['decay'], amsgrad=False)
    DNN.compile(optimizer='adam', loss='binary_crossentropy',\
                metrics=['accuracy'])
    
    # Train the DNN
    start = timer()
    logfilename = './models/'+target.replace('.','_')+'/training.log'
    DNN.fit(
        x_resampled, y_resampled, batch_size=params['batch_size'], epochs=200,\
        validation_data=(x_val,y_val), verbose=0,
        callbacks=[
            keras.callbacks.CSVLogger(logfilename, separator=',', append=True),
            keras.callbacks.EarlyStopping(patience=8,verbose=0,\
                                          restore_best_weights=True),
            keras.callbacks.ReduceLROnPlateau(factor=0.5,patience=3,verbose=0)
        ])
    train_time = timer() - start
    
    # Get predictions, calculate model performance and save info
    p_te = DNN.predict(x_te[rows_te])[:,0]
    y_testing=y_te[target][rows_te]
    auc_te = roc_auc_score(y_testing, p_te)
    print("%15s: %3.5f" % (target, auc_te))

    y_hat_testing=DNN.predict_classes(x_te[rows_te])
    average_precision=average_precision_score(y_testing,p_te)
    mv=evaluate_model_predictions(target,'DNN',0.5,y_testing,y_hat_testing,\
                                  auc_te,average_precision)
    filename = check_and_save(target,mv,DNN,True)

    # Find optimal decision threshold
    precision, recall, thresholds = precision_recall_curve(y_testing,p_te)
    # Find max MCC varying decision threshold, calculate _MaxMCC performance, save if best available
    mccs = []
    for th in thresholds:
        y_hat_testing_adj=adjusted_classes(p_te,th)
        mccs.append(matthews_corrcoef(y_testing,y_hat_testing_adj))
    m_idx = np.argmax(mccs)
    m_thresh = thresholds[m_idx]
    y_hat_testing_adj=adjusted_classes(p_te,m_thresh)
    mv=evaluate_model_predictions(target,'DNN_MaxMCC',m_thresh,y_testing,y_hat_testing_adj,\
                                  auc_te,average_precision)
    
    # Find max F1 varying probability threshold, calculate modified performance, save
    # f1 = 2*precision*recall/(precision+recall)  # Sometimes precision=recall=0!
#     p,r,t = zip(*[i for i in zip(precision,recall,np.append(thresholds,1)) if i[0:2]!=(0,0)])
#     p,r,t = np.array(p),np.array(r),np.array(t)
#     f1 = 2*p*r/(p+r)
#     m_idx = np.argmax(f1)
#     m_thresh = thresholds[m_idx]
#     y_hat_testing_adj=adjusted_classes(p_te,m_thresh)
#     mv=evaluate_model_predictions(target,'DNN_modT',m_thresh,y_testing,y_hat_testing_adj,\
#                                   auc_te,average_precision)
    if filename is None:
        check_and_save(target,mv,DNN,True)
    else:
        check_and_save(target,mv,filename,True)
    display(get_model_perfs(target))
    
    # Score to be used in evaluation
    
    score = mv['mcc']
    
    # Loss must be minimized
    
    loss = 1 - score
    
    # Write to the csv file ('a' means append)
    of_connection = open(out_file, 'a')
    writer = csv.writer(of_connection)
    writer.writerow([loss, params, ITERATION, train_time])
    
    return {'loss': loss, 'params': params, 'iteration': ITERATION,
            'train_time': train_time, 'status': STATUS_OK}

# Execution [possibly continuation] of the Optimization

In [7]:
for target in y_tr.columns[::-1]:
    # Set up trials object
    # Get previous trial data if it exists and initialize trials object
    # to be able to see our results after algorithm is complete
    target_path = './models/'+target.replace('.','_')+'/'
    if os.path.exists(os.path.join(target_path,'trials.pkl')) and Continue_Existing_Trials:
        trials = pickle.load(open(os.path.join(target_path,'trials.pkl'), "rb"))
        ITERATION = trials.results[-2]['iteration']
    else:
        trials = Trials()
        ITERATION = 0

    # Set up Tracking File
    out_file = os.path.join(target_path,'tracking.csv')
    if os.path.exists(out_file) and Continue_Existing_Trials:
        print(out_file,' ready for continued tracking.')
    else:
        set_up_tracking_file(out_file)
    
    # Determine rows with available data
    rows_tr = np.isfinite(y_tr[target]).values
    rows_te = np.isfinite(y_te[target]).values
    x,y = x_tr[rows_tr], y_tr[target][rows_tr]

    # Address Class Imbalance
    x_train, x_val, y_train, y_val = train_test_split(x, y, stratify=y, \
                                                      test_size=0.2, random_state=42)
    ros = RandomOverSampler(random_state=0)
    x_resampled, y_resampled = ros.fit_sample(x_train,y_train)

    # Optimize
    best = fmin(fn= objective,
                space= space,
                algo= tpe.suggest,
                max_evals = 250,
                trials= trials)
    print('Best model:',best)
    
    pickle.dump(trials, open(os.path.join(target_path,'trials.pkl'), "wb"))

  0%|          | 0/250 [00:00<?, ?it/s, best loss: ?]WARNING:tensorflow:From /home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where








         SR.p53: 0.78717                             
Model saved and metrics table updated.               
Metrics table updated with provided model filename.  
  0%|          | 0/250 [00:26<?, ?it/s, best loss: ?]

,model,threshold,accuracy,precision,recall,f1,auc_roc,mcc,avg_precision,confusion_matrix,model_filename
0,RF,0.500000,0.933442,0.500000,0.024390,0.046512,0.806702,0.099247,0.221990,"[[574, 1], [40, 1]]",RF0.joblib
1,RF_modT,0.220000,0.886364,0.254237,0.365854,0.300000,0.806702,0.245058,0.221990,"[[531, 44], [26, 15]]",RF0.joblib
2,DNN,0.500000,0.753247,0.150943,0.585366,0.240000,0.768696,0.199692,0.168234,"[[440, 135], [17, 24]]",DNN0.h5
3,DNN_modT,0.540362,0.780844,0.169014,0.585366,0.262295,0.768696,0.224981,0.168234,"[[457, 118], [17, 24]]",DNN0.h5
4,DNN,0.500000,0.654221,0.144628,0.853659,0.247350,0.787169,0.251947,0.196547,"[[368, 207], [6, 35]]",DNN1.h5
5,DNN_MaxMCC,0.529997,0.693182,0.160550,0.853659,0.270270,0.787169,0.279082,0.196547,"[[392, 183], [6, 35]]",DNN1.h5


         SR.p53: 0.78819                                                       
Model performance not better than that previously recorded.                    
Model performance not better than that previously recorded.                    
  0%|          | 1/250 [00:38<1:48:49, 26.22s/it, best loss: 0.720917540103331]

,model,threshold,accuracy,precision,recall,f1,auc_roc,mcc,avg_precision,confusion_matrix,model_filename
0,RF,0.500000,0.933442,0.500000,0.024390,0.046512,0.806702,0.099247,0.221990,"[[574, 1], [40, 1]]",RF0.joblib
1,RF_modT,0.220000,0.886364,0.254237,0.365854,0.300000,0.806702,0.245058,0.221990,"[[531, 44], [26, 15]]",RF0.joblib
2,DNN,0.500000,0.753247,0.150943,0.585366,0.240000,0.768696,0.199692,0.168234,"[[440, 135], [17, 24]]",DNN0.h5
3,DNN_modT,0.540362,0.780844,0.169014,0.585366,0.262295,0.768696,0.224981,0.168234,"[[457, 118], [17, 24]]",DNN0.h5
4,DNN,0.500000,0.654221,0.144628,0.853659,0.247350,0.787169,0.251947,0.196547,"[[368, 207], [6, 35]]",DNN1.h5
5,DNN_MaxMCC,0.529997,0.693182,0.160550,0.853659,0.270270,0.787169,0.279082,0.196547,"[[392, 183], [6, 35]]",DNN1.h5


         SR.p53: 0.72282                                                       
Model performance not better than that previously recorded.                    
Model performance not better than that previously recorded.                    
  1%|          | 2/250 [01:21<1:30:57, 22.01s/it, best loss: 0.720917540103331]

...

,model,threshold,accuracy,precision,recall,f1,auc_roc,mcc,avg_precision,confusion_matrix,model_filename
0,RF,0.500000,0.906077,0.615385,0.400000,0.484848,0.930331,0.447984,0.560184,"[[468, 15], [36, 24]]",RF0.joblib
1,RF_modT,0.382500,0.913444,0.600000,0.650000,0.624000,0.930331,0.575769,0.560184,"[[457, 26], [21, 39]]",RF0.joblib
2,DNN,0.500000,0.815838,0.330508,0.650000,0.438202,0.803882,0.369779,0.296565,"[[404, 79], [21, 39]]",DNN0.h5
3,DNN_modT,0.505391,0.817680,0.333333,0.650000,0.440678,0.803882,0.372498,0.296565,"[[405, 78], [21, 39]]",DNN0.h5
4,DNN_MaxMCC,0.626589,0.876611,0.444444,0.466667,0.455285,0.788371,0.385903,0.344479,"[[448, 35], [32, 28]]",DNN_MaxMCC0.h5
5,DNN,0.500000,0.779006,0.319277,0.883333,0.469027,0.908868,0.441899,0.598178,"[[370, 113], [7, 53]]",DNN1.h5
6,DNN_MaxMCC,0.758863,0.911602,0.596774,0.616667,0.606557,0.908868,0.556875,0.598178,"[[458, 25], [23, 37]]",DNN1.h5
7,DNN,0.500000,0.819521,0.366197,0.866667,0.514851,0.897792,0.485348,0.503395,"[[393, 90], [8, 52]]",DNN2.h5
8,DNN,0.500000,0.834254,0.386364,0.850000,0.531250,0.890114,0.498673,0.498337,"[[402, 81], [9, 51]]",DNN3.h5
9,DNN_MaxMCC,0.634612,0.885820,0.489583,0.783333,0.602564,0.902830,0.560364,0.519168,"[[434, 49], [13, 47]]",DNN_MaxMCC1.h5


         SR.MMP: 0.85454                                                             
Model performance not better than that previously recorded.                          
Model performance not better than that previously recorded.                          
 26%|██▋       | 66/250 [2:36:03<7:13:19, 141.30s/it, best loss: 0.43330530044289683]

,model,threshold,accuracy,precision,recall,f1,auc_roc,mcc,avg_precision,confusion_matrix,model_filename
0,RF,0.500000,0.906077,0.615385,0.400000,0.484848,0.930331,0.447984,0.560184,"[[468, 15], [36, 24]]",RF0.joblib
1,RF_modT,0.382500,0.913444,0.600000,0.650000,0.624000,0.930331,0.575769,0.560184,"[[457, 26], [21, 39]]",RF0.joblib
2,DNN,0.500000,0.815838,0.330508,0.650000,0.438202,0.803882,0.369779,0.296565,"[[404, 79], [21, 39]]",DNN0.h5
3,DNN_modT,0.505391,0.817680,0.333333,0.650000,0.440678,0.803882,0.372498,0.296565,"[[405, 78], [21, 39]]",DNN0.h5
4,DNN_MaxMCC,0.626589,0.876611,0.444444,0.466667,0.455285,0.788371,0.385903,0.344479,"[[448, 35], [32, 28]]",DNN_MaxMCC0.h5
5,DNN,0.500000,0.779006,0.319277,0.883333,0.469027,0.908868,0.441899,0.598178,"[[370, 113], [7, 53]]",DNN1.h5
6,DNN_MaxMCC,0.758863,0.911602,0.596774,0.616667,0.606557,0.908868,0.556875,0.598178,"[[458, 25], [23, 37]]",DNN1.h5
7,DNN,0.500000,0.819521,0.366197,0.866667,0.514851,0.897792,0.485348,0.503395,"[[393, 90], [8, 52]]",DNN2.h5
8,DNN,0.500000,0.834254,0.386364,0.850000,0.531250,0.890114,0.498673,0.498337,"[[402, 81], [9, 51]]",DNN3.h5
9,DNN_MaxMCC,0.634612,0.885820,0.489583,0.783333,0.602564,0.902830,0.560364,0.519168,"[[434, 49], [13, 47]]",DNN_MaxMCC1.h5


 27%|██▋       | 67/250 [2:37:15<7:09:32, 140.83s/it, best loss: 0.43330530044289683]


KeyboardInterrupt: 